In [0]:
from functions import *
from inputs import *
from pyspark.sql import functions as F

In [0]:
source = 'bookstore.bronze.books_temp'
target = 'bookstore.silver.books'

display(spark.sql(f"select * from {source}").sort('book_id','updated'))
display(spark.sql(f"select * from {target}").sort('book_id','start_date'))

In [0]:
def read_file(file_name,name_,schema_):
    return (
        topics_feed(feed=raw_feed(spark=spark
                                  ,path_='s3://bookstorerawdata/rawfeed/Inputs/'
                                  ,schema_=schema_feeds
                                  ,lakeflow=0)
                    ,name_=name_
                    ,schema_=schema_)
        .filter(F.col('source_file') == file_name)
    )

feed_01 = read_file('s3://bookstorerawdata/rawfeed/Inputs/feed_01.json',topic_books,schema_books)
display(feed_01.sort("book_id"))
# feed_01.write.mode("append").saveAsTable(source)

AK_booksUpdate_01 = read_file('s3://bookstorerawdata/rawfeed/Inputs/AK_booksUpdate_01.json',topic_books,schema_books)
display(AK_booksUpdate_01.sort("book_id"))
# AK_booksUpdate_01.write.mode("append").saveAsTable(source)

AK_booksUpdate_02 = read_file('s3://bookstorerawdata/rawfeed/Inputs/AK_booksUpdate_02.json',topic_books,schema_books)
display(AK_booksUpdate_02.sort("book_id"))
# AK_booksUpdate_02.write.mode("append").saveAsTable(source)

feed_01_customers = read_file('s3://bookstorerawdata/rawfeed/Inputs/feed_01.json',topic_customers,schema_customers)
feed_02_customers = read_file('s3://bookstorerawdata/rawfeed/Inputs/feed_02.json',topic_customers,schema_customers)
display(feed_01_customers.sort('customer_id'))
display(feed_02_customers.sort('customer_id'))

# books_scdType2_upsert(spark=spark
#                       ,source=source
#                       ,target=target)

In [0]:
df = feed_01_customers.union(feed_02_customers)

from pyspark.sql import Window
w = Window.partitionBy("customer_id").orderBy(F.col("row_time").desc())
ranked_df = (
    df
    .withColumn("rank",F.rank().over(w))
    .filter("rank == 1")
    .drop("rank")
       )

display(df.groupBy("source_file").count().orderBy("source_file")) #300,300
display(feed_01_customers.dropDuplicates(['customer_id']).count()) #207
display(feed_02_customers.dropDuplicates(['customer_id']).count()) #205
display(ranked_df.count()) #396
display(ranked_df.groupBy("source_file").count().orderBy("source_file")) #191,205

display(df.orderBy(["customer_id","row_time"],ascending=[True,False]))
display(ranked_df.orderBy(["customer_id","row_time"],ascending=[True,False]))
display(spark.read.table(tbl_bronze_customers).orderBy(["customer_id","row_time"],ascending=[True,False]))



In [0]:
%sql
-- drop table if exists bookstore.bronze.books_temp;
-- CREATE TABLE IF NOT EXISTS bookstore.bronze.books_temp
-- (book_id STRING,title STRING,author STRING,price DOUBLE,updated TIMESTAMP,source_file STRING,insert_ts TIMESTAMP);
-- truncate table bookstore.bronze.books_temp;
-- truncate table bookstore.silver.books;